In [1]:
# Step 1: basic setup

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Pandas display options
pd.set_option("display.max_rows", 20)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

# Confirm working directory
print("Working directory:", os.getcwd())


Working directory: c:\Users\Adam Nhan\prediction-market-money-printer\notebooks
